Binary Classification 'Label'

In [3]:
!pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch

c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import pandas as pd
url = 'https://github.com/bvidgen/Dynamically-Generated-Hate-Speech-Dataset/raw/refs/heads/main/Dynamically%20Generated%20Hate%20Dataset%20v0.2.3.csv'
df = pd.read_csv(url)
df.head(5)

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
4,5,acl10000,5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,1,NaN


In [6]:
df['label'] = df['label'].map({'hate': 1, 'nothate': 0})

In [7]:
df1 = df[['text','label','split']]
df1.head(5)

,text,label,split
0,dalits are lowlives,1,train
1,gay people are a burden to our society,1,train
2,Arabs are not welcome,1,train
3,I'm not saying we should actually eliminate he...,1,train
4,bananas are for black people,1,test


In [8]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Example data
texts = df1['text']
labels = df1['label'] # Binary labels
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128

dataset = TextDataset(texts, labels, tokenizer, max_len)


In [9]:
train_texts = df1[df1['split'] == 'train']['text'].tolist()
train_labels = df1[df1['split'] == 'train']['label'].tolist()
val_texts = df1[df1['split'] == 'dev']['text'].tolist()
val_labels = df1[df1['split'] == 'dev']['label'].tolist()
test_texts = df1[df1['split'] == 'test']['text'].tolist()
test_labels = df1[df1['split'] == 'test']['label'].tolist()

# Assuming TextDataset is your custom dataset class
train_dataset = TextDataset(train_texts, train_labels, tokenizer, max_len)
val_dataset = TextDataset(val_texts, val_labels, tokenizer, max_len)
test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_len)

In [26]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 69%|██████▉   | 8541/12348 [7:20:07<3:16:10,  3.09s/it]
                                                    
  0%|          | 10/12348 [00:15<4:39:50,  1.36s/it]    

{'loss': 0.7172, 'grad_norm': 3.1423232555389404, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


                                                    
  0%|          | 20/12348 [00:28<4:39:28,  1.36s/it]    

{'loss': 0.6893, 'grad_norm': 4.155902862548828, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


                                                    
  0%|          | 30/12348 [00:43<5:00:41,  1.46s/it]    

...

                                                       
100%|█████████▉| 12340/12348 [4:47:28<00:11,  1.39s/it] 

{'loss': 0.476, 'grad_norm': 8.183664321899414, 'learning_rate': 3.3760972316002705e-08, 'epoch': 3.0}


                                                       
100%|██████████| 12348/12348 [4:47:40<00:00,  1.40s/it] 

{'train_runtime': 17260.0469, 'train_samples_per_second': 5.723, 'train_steps_per_second': 0.715, 'train_loss': 0.40006953920018995, 'epoch': 3.0}


TrainOutput(global_step=12348, training_loss=0.40006953920018995, metrics={'train_runtime': 17260.0469, 'train_samples_per_second': 5.723, 'train_steps_per_second': 0.715, 'total_flos': 6497001290004480.0, 'train_loss': 0.40006953920018995, 'epoch': 3.0})

In [27]:
model.save_pretrained('./results')

In [28]:
tokenizer.save_pretrained('./results')

('./results\\tokenizer_config.json',
 './results\\special_tokens_map.json',
 './results\\vocab.txt',
 './results\\added_tokens.json')

In [2]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('./results')
tokenizer = BertTokenizer.from_pretrained('./results')

c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained('./results')
tokenizer = BertTokenizer.from_pretrained('./results')

# Prepare your input text
text = "Input your text here"
inputs = tokenizer(text, return_tensors='pt')

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to probabilities
probabilities = torch.nn.functional.softmax(logits, dim=-1)

# Get the predicted class
predicted_class = torch.argmax(probabilities, dim=1).item()

print(f"Predicted class: {predicted_class}")

Predicted class: 1


In [11]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load the trained model
model = BertForSequenceClassification.from_pretrained('./results')
model.eval()

# Create DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

def evaluate_model(model, dataloader):
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            
            outputs = model(input_ids=inputs, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return all_preds, all_labels

def test_model():
    preds, labels = evaluate_model(model, test_loader)
    macro_f1 = f1_score(labels, preds, average='macro')
    print(f'Macro F1 Score: {macro_f1}')

test_model()

Macro F1 Score: 0.7640764024258563
